---
title: "4. Manipulating Geospatial Data"
author: "kim dayeon"
date: today
categories: [news, code, analysis]
image: "E1.png"
---

# Introduction

You are a Starbucks big data analyst ([that’s a real job!](https://www.forbes.com/sites/bernardmarr/2018/05/28/starbucks-using-big-data-analytics-and-artificial-intelligence-to-boost-performance/#130c7d765cdc)) looking to find the next store into a [Starbucks Reserve Roastery](https://www.businessinsider.com/starbucks-reserve-roastery-compared-regular-starbucks-2018-12#also-on-the-first-floor-was-the-main-coffee-bar-five-hourglass-like-units-hold-the-freshly-roasted-coffee-beans-that-are-used-in-each-order-the-selection-rotates-seasonally-5).  These roasteries are much larger than a typical Starbucks store and have several additional features, including various food and wine options, along with upscale lounge areas.  You'll investigate the demographics of various counties in the state of California, to determine potentially suitable locations.

<center>
<img src="https://storage.googleapis.com/kaggle-media/learn/images/BIyE6kR.png" width="450"><br/><br/>
</center>

Before you get started, run the code cell below to set everything up.

In [15]:
import math
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim            # What you'd normally run
#from learntools.geospatial.tools import Nominatim # Just for this exercise

import folium 
from folium import Marker
from folium.plugins import MarkerCluster


#from learntools.core import binder
#binder.bind(globals())
#from learntools.geospatial.ex4 import *

In [11]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

# Exercises

### 1) Geocode the missing locations.

Run the next code cell to create a DataFrame `starbucks` containing Starbucks locations in the state of California.

In [12]:
# Load and preview Starbucks locations in California
starbucks = pd.read_csv("D:/archive (1)/starbucks_locations.csv")
starbucks.head()

,Store Number,Store Name,Address,City,Longitude,Latitude
0,10429-100710,Palmdale & Hwy 395,14136 US Hwy 395 Adelanto CA,Adelanto,-117.40,34.51
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15
3,29839-255026,Target Anaheim T-0677,8148 E SANTA ANA CANYON ROAD AHAHEIM CA,AHAHEIM,-117.75,33.87
4,23463-230284,Safeway - Alameda 3281,2600 5th Street Alameda CA,Alameda,-122.28,37.79


* 대부분의 상점에는 알려진 위치(위도, 경도)가 있습니다. 그러나 Berkeley 시의 모든 위치가 누락되었습니다.

In [13]:
# How many rows in each column have missing values?
print(starbucks.isnull().sum())

# View rows with missing locations
rows_with_missing = starbucks[starbucks["City"]=="Berkeley"]
rows_with_missing

Store Number    0
Store Name      0
Address         0
City            0
Longitude       5
Latitude        5
dtype: int64


,Store Number,Store Name,Address,City,Longitude,Latitude
153,5406-945,2224 Shattuck - Berkeley,2224 Shattuck Avenue Berkeley CA,Berkeley,NaN,NaN
154,570-512,Solano Ave,1799 Solano Avenue Berkeley CA,Berkeley,NaN,NaN
155,17877-164526,Safeway - Berkeley #691,1444 Shattuck Place Berkeley CA,Berkeley,NaN,NaN
156,19864-202264,Telegraph & Ashby,3001 Telegraph Avenue Berkeley CA,Berkeley,NaN,NaN
157,9217-9253,2128 Oxford St.,2128 Oxford Street Berkeley CA,Berkeley,NaN,NaN


튜토리얼에서 우리는 (geopy.geocoders의) Nominatim()을 사용하여 값을 지오코딩했으며 이는 이 과정 외부에서 자신의 프로젝트에서 사용할 수 있는 것입니다.

이 연습에서는 약간 다른 함수 Nominatim()(learntools.geospatial.tools에서)을 사용합니다. 이 기능은 노트북 상단에서 가져온 기능으로 GeoPandas의 기능과 동일하게 작동합니다.

*즉, 다음과 같은 경우:

노트북 상단의 import 문을 변경하지 않고
아래 코드 셀에서 지오코딩 함수를 geocode()로 호출합니다.


In [23]:
# Create the geocoder
geolocator = Nominatim(user_agent="kaggle_learn")

def my_geocoder(row):
    point = geolocator.geocode(row).point
    return pd.Series({'Latitude': point.latitude, 'Longitude': point.longitude})

berkeley = rows_with_missing.apply(lambda x: my_geocoder(x['Address']), axis=1)
starbucks.update(berkeley)


# 2) View Berkeley locations.
OpenStreetMap 스타일로 Berkeley의 (위도, 경도) 위치를 시각화합니다.

In [50]:
# Create a base map
m_2= folium.Map(location=[37.88,-122.26], tiles='openstreetmap',zoom_start=13)

for idx, row in starbucks[starbucks["City"]=='Berkeley'].iterrows():
    Marker([row['Latitude'], row['Longitude']]).add_to(m_2)

m_2


# 3) Consolidate your data.
아래 코드를 실행하여 캘리포니아 주의 각 카운티에 대한 이름, 지역(평방 킬로미터) 및 고유 ID("GEOID" 열에 있음)가 포함된 GeoDataFrame CA_counties를 로드합니다. "도형" 열에는 카운티 경계가 있는 다각형이 포함되어 있습니다.

In [25]:
CA_counties = gpd.read_file("D:/archive (1)/CA_county_boundaries/CA_county_boundaries/CA_county_boundaries.shp")
CA_counties.crs = {'init': 'epsg:4326'}
CA_counties.head()

c:\Users\jiyeo\anaconda3\envs\dayeon\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,GEOID,name,area_sqkm,geometry
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6..."
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7..."
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822..."
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3..."
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360..."


* CA_pop에는 각 카운티의 인구 추정치가 포함되어 있습니다.
* CA_high_earners는 연간 소득이 $150,000 이상인 가구 수를 포함합니다.
* CA_median_age에는 각 카운티의 평균 연령이 포함됩니다.

In [31]:
CA_pop = pd.read_csv("D:/archive (1)/CA_county_population.csv", index_col="GEOID")
CA_high_earners = pd.read_csv("D:/archive (1)/CA_county_high_earners.csv", index_col="GEOID")
CA_median_age = pd.read_csv("D:/archive (1)/CA_county_median_age.csv", index_col="GEOID")

* CA_pop, CA_high_earners 및 CA_median_age와 함께 CA_counties GeoDataFrame을 조인하기

결과 GeoDataFrame CA_stats의 이름을 지정하고 "GEOID", "name", "area_sqkm", "geometry", "population", "high_earners" 및 "median_age"의 8개 열이 있는지 확인합니다.

In [38]:
a=CA_counties.merge(CA_pop, on = "GEOID")
ab = a.merge(CA_high_earners,on = "GEOID")
CA_stats = ab.merge(CA_median_age,on = "GEOID")
CA_stats.head()


,GEOID,name,area_sqkm,geometry,population,high_earners,median_age
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6...",2987,111,55.0
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7...",1540975,65768,35.9
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822...",446527,25231,33.7
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3...",45602,2046,51.6
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360...",850967,57121,37.5


In [41]:
CA_stats["density"] = CA_stats["population"] / CA_stats["area_sqkm"]
CA_stats.head()

,GEOID,name,area_sqkm,geometry,population,high_earners,median_age,density
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6...",2987,111,55.0,1.198638
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7...",1540975,65768,35.9,598.376878
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822...",446527,25231,33.7,45.499825
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3...",45602,2046,51.6,16.980022
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360...",850967,57121,37.5,148.788107


# 4) Which counties look promising?
모든 정보를 단일 GeoDataFrame으로 축소하면 특정 기준을 충족하는 카운티를 훨씬 쉽게 선택할 수 있습니다.

다음 코드 셀을 사용하여 CA_stats GeoDataFrame에서 행(및 모든 열)의 하위 집합을 포함하는 GeoDataFrame sel_counties를 생성합니다. 특히 다음과 같은 카운티를 선택해야 합니다.

연간 $150,000를 버는 가구가 100,000가구 이상이며,
평균 연령은 38.5세 미만이며,
주민 밀도는 최소 285(평방 킬로미터당)입니다.
또한 선택한 카운티는 다음 기준 중 하나 이상을 충족해야 합니다.

연간 $150,000를 버는 가구가 500,000가구 이상이며,
평균 연령이 35.5세 미만이거나
주민 밀도는 최소 1400(평방 킬로미터당)입니다.

In [42]:
sel_counties= CA_stats[(CA_stats.high_earners >= 100000) & (CA_stats.median_age < 38.5) & (CA_stats.density>285) &
         (CA_stats.high_earners >= 500000) & (CA_stats.median_age < 35.5) | (CA_stats.density>1400)]

# 5) How many stores did you identify?
다음 Starbucks Reserve Roastery 위치를 찾을 때 선택한 카운티 내의 모든 매장을 고려하고 싶을 것입니다. 그렇다면 선택한 카운티 내에 몇 개의 매장이 있습니까?

이 질문에 답할 준비를 하려면 다음 코드 셀을 실행하여 모든 starbucks 위치가 포함된 GeoDataFrame starbucks_gdf를 만듭니다.

In [44]:
starbucks_gdf = gpd.GeoDataFrame(starbucks, geometry=gpd.points_from_xy(starbucks.Longitude, starbucks.Latitude))
starbucks_gdf.crs = {'init': 'epsg:4326'}
starbucks_gdf.head()

c:\Users\jiyeo\anaconda3\envs\dayeon\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,Store Number,Store Name,Address,City,Longitude,Latitude,geometry
0,10429-100710,Palmdale & Hwy 395,14136 US Hwy 395 Adelanto CA,Adelanto,-117.40,34.51,POINT (-117.40000 34.51000)
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16,POINT (-118.76000 34.16000)
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15,POINT (-118.76000 34.15000)
3,29839-255026,Target Anaheim T-0677,8148 E SANTA ANA CANYON ROAD AHAHEIM CA,AHAHEIM,-117.75,33.87,POINT (-117.75000 33.87000)
4,23463-230284,Safeway - Alameda 3281,2600 5th Street Alameda CA,Alameda,-122.28,37.79,POINT (-122.28000 37.79000)


In [49]:
locations= gpd.sjoin(starbucks_gdf, sel_counties)
num_stores = len(locations)
num_stores

87

# 6) Visualize the store locations.
이전 질문에서 식별한 매장의 위치를 ​​표시하는 지도를 만듭니다.

In [55]:
m_6 = folium.Map(location=[37,-120], zoom_start=6)


mc = MarkerCluster()

locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties)
for idx, row in locations_of_interest.iterrows():
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc.add_child(folium.Marker([row['Latitude'], row['Longitude']]))

m_6.add_child(mc)


